In [82]:
#import the modules
import os
import numpy as np
import pandas as pd
import plotly as plt 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt1

In [56]:
import time
import google.cloud
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters

In [57]:
plant1_pg = pd.read_csv("../Data/Plant_1_Generation_Data.csv")
plant2_pg = pd.read_csv("../Data/Plant_1_Weather_Sensor_Data.csv")
plant1_ws = pd.read_csv("../Data/Plant_2_Generation_Data.csv")
plant2_ws = pd.read_csv("../Data/Plant_2_Weather_Sensor_Data.csv")

#creando un mapa
files = {
    0: plant1_pg,
    1: plant1_ws,
    2: plant2_pg,
    3: plant2_ws,
}

In [58]:
#datos de generación de energía de la planta 1
files[0].sample(5)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
18717,24-05-2020 18:00,4135001,VHMLBKoKgIrUVDU,626.0,60.6125,8159.750,7280491.750
58142,12-06-2020 22:30,4135001,iCRJl6heRkivqQ3,0.0,0.0000,5888.000,7391359.000
41020,04-06-2020 18:15,4135001,ZnxXDlPa8U1GXgE,295.0,28.4875,8173.750,6679057.750
19872,25-05-2020 07:45,4135001,3PZuoBAID5Wc2HD,3691.5,362.2750,287.875,7061855.875
43279,05-06-2020 21:00,4135001,VHMLBKoKgIrUVDU,0.0,0.0000,0.000,7371554.000


In [59]:
#datos del sensor meteorológico de la planta 1
files[1].sample(5)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
59310,2020-06-14 00:30:00,4136001,oZZkBaNadn6DNKz,0.000000,0.000000,0.000000,1.708272e+09
5118,2020-05-17 10:15:00,4136001,mqwcsP2rE7J0TFp,739.000000,723.566667,2018.200000,5.935993e+08
43952,2020-06-06 18:00:00,4136001,mqwcsP2rE7J0TFp,103.714286,100.800000,8324.142857,5.937510e+08
42408,2020-06-06 00:30:00,4136001,Qf4GUc1pJu5T6c6,0.000000,0.000000,7599.000000,8.385822e+08
1942,2020-05-15 22:00:00,4136001,LlT2YUhhzqhg5Sw,0.000000,0.000000,9234.000000,2.826020e+08


In [60]:
#datos de generación de energía de la planta 2
files[2].sample(6)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
2444,2020-06-10 07:30:00,4135001,HmiyD2TTLFNqkNe,23.833986,30.019338,0.281592
2697,2020-06-12 22:45:00,4135001,HmiyD2TTLFNqkNe,22.599839,21.062392,0.000000
781,2020-05-23 19:00:00,4135001,HmiyD2TTLFNqkNe,27.826436,26.217763,0.000000
2145,2020-06-07 04:45:00,4135001,HmiyD2TTLFNqkNe,21.940753,20.406875,0.000000
2097,2020-06-06 16:45:00,4135001,HmiyD2TTLFNqkNe,25.334819,27.200340,0.081607
1557,2020-06-01 01:30:00,4135001,HmiyD2TTLFNqkNe,22.348948,20.201571,0.000000


In [69]:
#datos del sensor meteorológico de la planta 2
files[3].sample(7)

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
2875,2020-06-14 00:00:00,iq8k7ZNt4Mwm3w0,23.775726,22.117714,0.000000
3126,2020-06-16 14:45:00,iq8k7ZNt4Mwm3w0,30.503410,41.550000,0.447018
3213,2020-06-17 12:30:00,iq8k7ZNt4Mwm3w0,28.204097,37.502086,0.470896
1845,2020-06-03 06:15:00,iq8k7ZNt4Mwm3w0,23.793487,22.963337,0.019233
762,2020-05-22 23:00:00,iq8k7ZNt4Mwm3w0,27.658895,25.563916,0.000000
3083,2020-06-16 04:00:00,iq8k7ZNt4Mwm3w0,24.040096,22.555920,0.000000
1165,2020-05-27 03:45:00,iq8k7ZNt4Mwm3w0,25.990678,24.988451,0.000000


In [ ]:
INSTANCE_ID = 'table-1'
TABLE_ID = 'orders_{}'.format(time.time())

In [ ]:
TABLE_ID

### Connecting to BigTable

In [ ]:
client = bigtable.Client.from_service_account_json('../Data/unbosque-service-account.json', admin = True)

In [ ]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

### Creating a table


In [ ]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

In [ ]:
print('Creating all family columns')

customer_cfId = 'customer'
customer_cf = table.column_family(customer_cfId)
customer_cf.create()

items_cfId = 'items'
items_cf = table.column_family(items_cfId)
items_cf.create()

delivery_cfId = 'delivery'
delivery_cf = table.column_family(delivery_cfId)
delivery_cf.create()

### Inserting data de los csv cargados previamente

In [62]:
for i in range(len(files)):
    files[i]["DATE_TIME"] = pd.to_datetime(files[i]["DATE_TIME"])
    files[i] =  files[i].drop(columns=["PLANT_ID"], axis=1)

In [63]:
import plotly.express as px

#Metodo para crear la grafica inicial de la consulta por la clave
def LineChart(temp_df,columns,start_date_time,end_date_time, title):
    temp_df = temp_df.loc[start_date_time : end_date_time]
    fig = px.line(temp_df[columns])
    fig.update_layout(title_text = title, title_x=0.5)
    fig.show()

### Consulta 1

In [95]:
data0 = files[3][files[3].SOURCE_KEY == "iq8k7ZNt4Mwm3w0"]

#x_values = data0['AMBIENT_TEMPERATURE']
#y_values = data0['MODULE_TEMPERATURE']
#plt1.bar(x_values, y_values)
data0= data0.set_index('DATE_TIME')

s1time = "01-06-2020 06:00"
e1time = "01-06-2020 12:00"
LineChart(data0, ["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE"],s1time,e1time,"Medidas de temperatura ambiente para la planta 2 ")


### Consulta 2

In [72]:
data1 = files[0][files[0].SOURCE_KEY == "adLQvlD726eNBSB"]
data1= data1.set_index('DATE_TIME')

stime = "15-05-2020 2:15"
etime = "15-05-2020 20:00"
LineChart(data1, ["DC_POWER","AC_POWER"],stime,etime,"Power Generation during para las 2:15 PM del 15 de mayo")

### Creating a table

In [7]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the orders_1634687023.5233824 table


In [8]:
print('Creating all family columns')

customer_cfId = 'customer'
customer_cf = table.column_family(customer_cfId)
customer_cf.create()

items_cfId = 'items'
items_cf = table.column_family(items_cfId)
items_cf.create()

delivery_cfId = 'delivery'
delivery_cf = table.column_family(delivery_cfId)
delivery_cf.create()

Creating all family columns


### Inserting data

In [ ]:
orders = [
    {
        'orderId': '2497',
        'customer': {
            'firstName': 'Fabian',
            'lastName': 'Penia',
            'customerId': '101'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '5',
                'unitPrice': '12000'
            },
            {
                'productId': '3302',
                'quantity': '1',
                'unitPrice': '18000'
            },
            {
                'productId': '3303',
                'quantity': '2',
                'unitPrice': '25000'
            }
        ],
        'delivery': {
            'addressLine1': 'Fake Street 123',
            'addressLine2': 'House 10',
            'city': 'Springfield',
            'country': 'US'
        }
    },
    {
        'orderId': '2498',
        'customer': {
            'firstName': 'Pepito',
            'lastName': 'Perez',
            'customerId': '102'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '1',
                'unitPrice': '12000'
            },
            {
                'productId': '3306',
                'quantity': '3',
                'unitPrice': '20000'
            }
        ],
        'delivery': {
            'addressLine1': '742 Evergreen Terrace',
            'city': 'Springfield',
            'country': 'US'
        }
    }
]

In [ ]:
dt = dt.utcnow()
rows = []

print('Writing orders to the table')
for order in orders:
    
    row_key = 'order#{}'.format(order['orderId']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(customer_cfId, 'firstName'.encode(), order['customer']['firstName'], timestamp = dt)
    row.set_cell(customer_cfId, 'lastName'.encode(), order['customer']['lastName'], timestamp = dt)
    row.set_cell(customer_cfId, 'customerId'.encode(), order['customer']['customerId'], timestamp = dt)
    
    for item in order['items']:
        row.set_cell(items_cfId, 'product#{}'.format(item['productId']).encode(), (item['quantity'] + ':' + item['unitPrice']), timestamp = dt)
    
    row.set_cell(delivery_cfId, 'addressLine1'.encode(), order['delivery']['addressLine1'], timestamp = dt)
    try:
        row.set_cell(delivery_cfId, 'addressLine2'.encode(), order['delivery']['addressLine2'], timestamp = dt)
    except KeyError:
        pass
    row.set_cell(delivery_cfId, 'city'.encode(), order['delivery']['city'], timestamp = dt)
    row.set_cell(delivery_cfId, 'country'.encode(), order['delivery']['country'], timestamp = dt)
    
    rows.append(row)

table.mutate_rows(rows)

### Reading data

In [ ]:
print('Getting a single order by row key')
key = 'order#2497'.encode()

row = table.read_row(key)
print_row(row)

In [ ]:
print('Getting a specific product item by row key')
key = 'order#2497'.encode()

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())

row = table.read_row(key, filter_ = col_filter)
print_row(row)

In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

In [ ]:
print('Getting a specific column for all orders')

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())

rows = table.read_rows(filter_ = col_filter)
for row in rows:
    print_row(row)

### Deleting a row

In [ ]:
print('Deleting a row by key')
key = 'order#2498'.encode()

row = table.direct_row(row_key)
row.delete()
row.commit()

In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

### Deleting table

In [ ]:
print('Deleting the {} table.'.format(TABLE_ID))
table.delete()

### Functions

In [ ]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

In [ ]:
firstName = row.cells[customer_cfId]['firstName'.encode()][0].value.decode('utf-8')
lastName = row.cells[customer_cfId]['lastName'.encode()][0].value.decode('utf-8')
customerId = row.cells[customer_cfId]['customerId'.encode()][0].value.decode('utf-8')

print(firstName + ' | ' + lastName + ' | ' + customerId)
print("\n")

for item in [k.decode('utf-8') for k in row.cells[items_cfId].keys()]:
    cell_content = row.cells[items_cfId][item.encode()][0].value.decode('utf-8')
    print(item + ' | ' + ' | '.join(cell_content.split(':')))
    
print("\n")

addressLine1 = row.cells[delivery_cfId]['addressLine1'.encode()][0].value.decode('utf-8')
addressLine2 = row.cells[delivery_cfId]['addressLine2'.encode()][0].value.decode('utf-8')
city = row.cells[delivery_cfId]['city'.encode()][0].value.decode('utf-8')
country = row.cells[delivery_cfId]['country'.encode()][0].value.decode('utf-8')

print(addressLine1 + ' | ' + addressLine2 + ' | ' + city + ' | ' + country)